# Exploratory Data Analysis of Zillow Data Set

This data is from the Kaggle competition to improve Zillow's "Zestimate": https://www.kaggle.com/c/zillow-prize-1

The data:

* properties_2017.csv: a sample of all properties from 2017 listed on Zillow through Sept
* properties_2016.csv: a sample of all properties from 2016 listed on Zillow
* train_2017.csv: contains dates, propertyids, and logerror for each transaction in 2017 through Sept
* train_2016_v2.csv: contains dates, propertyids, and logerror for each transaction in 2016
* Not all properties have transactions
* logerror=log(Zestimate)−log(SalePrice)

Goal: find a model that reduces the logerror

In [ ]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
from matplotlib.pyplot import cm
from uszipcode import ZipcodeSearchEngine

In [ ]:
pd.set_option('display.max_columns', 65)

# Data Ingestion

In [ ]:
df16 = pd.read_csv('properties_2016.csv', low_memory=False)
df_transactions16 = pd.read_csv('train_2016_v2.csv', low_memory=False)
df_merged16 = pd.merge(df16, df_transactions16, on='parcelid', how='right')
df_merged16.head()

In [ ]:
df17 = pd.read_csv('properties_2017.csv', low_memory=False)
df_transactions17 = pd.read_csv('train_2017.csv', low_memory=False)
df_merged17 = pd.merge(df17, df_transactions17, on='parcelid', how='right')
df_merged17.head()

## Data Cleaning for Exploration

Looking at the data above, a few columns need to be cleaned up before we can do exploratory data analysis.

First, the latitudes and longitudes are missing their decimal points:

In [ ]:
df_merged17['latitude'] = df_merged17['latitude'] / 1000000
df_merged17['longitude'] = df_merged17['longitude'] / 1000000
df_merged16['latitude'] = df_merged16['latitude'] / 1000000
df_merged16['longitude'] = df_merged16['longitude'] / 1000000

Second, the tax delinquency years are listed as YY, with the first digit missing if it is a 0. Since some of the years could be from the previous century, we need to fix this so that they will sort in the correct order:

In [ ]:
def convertyears(x):
    if x > 9 and x < 20:
        t = '20' + str(x)
        return float(t)
    elif x <= 9:
        t = '200' + str(x)
        return float(t)
    elif x > 20:
        t = '19' + str(x)
        return float(t)
    else:
        return np.nan
    


df_merged17['taxdelinquencyyear'] = df_merged17['taxdelinquencyyear'].map(lambda a: convertyears(a))
df_merged16['taxdelinquencyyear'] = df_merged16['taxdelinquencyyear'].map(lambda a: convertyears(a))

Let's also do a quick check of data types:

In [ ]:
df_merged16.dtypes

The transaction dates will be more useful in a datetime format. Categorical columns also have the wrong types, but we will deal with those on a case by case basis.

In [ ]:
format = '%Y-%m-%d'
df_merged16['transactiondate'] = df_merged16['transactiondate'].map(lambda a: datetime.datetime.strptime(a, format))
df_merged17['transactiondate'] = df_merged17['transactiondate'].map(lambda a: datetime.datetime.strptime(a, format))

The pool types have been separated into one-hot columns. We are going to combine them, calculating based on poolcnt and hashottuborspa.

In [ ]:
def pooltypes(a, b):
    if a and b > 0:
        return 2
    elif not(a) and b > 0:
        return 7
    elif a and b == 0:
        return 10
    else:
        return 0

    
df_total['poolcnt'].fillna(0, inplace=True)
df_total['hashottuborspa'].fillna(False, inplace=True)

df_merged16['pooltype'] = df_merged16.apply(lambda x: pooltypes(x['hashottuborspa'], x['poolcnt']), axis=1)
df_merged17['pooltype'] = df_merged17.apply(lambda x: pooltypes(x['hashottuborspa'], x['poolcnt']), axis=1)

In some analyses we will be looking at the combined data sets:

In [ ]:
df_merged16['setyear'] = 2016
df_merged17['setyear'] = 2017
df_total = df_merged16.append(df_merged17, ignore_index=True)
df_total.head()

# Location Exploration

## Map The Log Errors

In [ ]:
sns.set()
plt.figure(figsize=(12,12))
sns.jointplot(x=df_total.latitude.values, y=df_total.longitude.values, size=10)
plt.ylabel('Longitude', fontsize=12)
plt.xlabel('Latitude', fontsize=12)
plt.show()

## Location Features

The location features are:
* latitude
* longitude
* regionidzip
* regionidcity
* regionidcounty
* regionidneighborhood
* fips
* censustractandblock
* rawcensustractandblock

Let's start with FIPS code, a federal code system for counties:

In [ ]:
df_total['fips'].value_counts()

To look up FIPS codes: https://www.census.gov/geo/reference/codes/cou.html

Counties in this set:
* 6037: LA County 
* 6059: Orange County 
* 6111: Ventura County

These should map 1:1 to regionidcounty values

In [ ]:
pd.crosstab(df_total['fips'],df_total['regionidcounty'])

The two values correspond, so in our feature selection we will use FIPS since that has real-world meaning.

Next, we'll look at the ZIP code data:

In [ ]:
df_total['regionidzip'].describe()

There appears to be an invalid US zip code for the max. Examine all impossible US zip codes:

In [ ]:
temp = df_total[df_total['regionidzip'] > 100000]
temp['regionidzip']

All of the entries have the same invalid zip. Look at the county the zip code is associated with.

In [ ]:
temp['fips'].value_counts()

All have the same county. Get all entries in that county:

In [ ]:
temp2 = df_total[df_total['fips'] == 6037]
temp2['regionidzip'].describe()

The zip code is most likely a military zip code. Let's look at some other features of the set:

In [ ]:
temp2['regionidzip'].mode()

This is not a US zip code. In spot checking, some of the zip codes are from CA, some are from OR, and some don't exist. In our data cleaning we will need to replace the zip code column. 

Look at the other region identifiers:

In [ ]:
df_total['regionidcity'].describe()

In [ ]:
df_total['regionidneighborhood'].describe()

In [ ]:
nbcorr = df_total[df_total['fips']==6111]

search = ZipcodeSearchEngine()
zips = pd.DataFrame(columns=['parcelid','zipcode'])

for i, row in nbcorr.iterrows():
    b = search.by_coordinate(row['latitude'],row['longitude'])
    zips.loc[len(zips)] = [row['parcelid'],b[0].Zipcode]
df_neighborhoods = pd.merge(nbcorr, zips, on='parcelid', how='left')
df_neighborhoods.groupby('regionidneighborhood')['regionidzip'].value_counts()

Most neighborhoods are contained within a single zip code, but some cross zipcode boundaries.

Finally, lets look at the census tracts.

In [ ]:
df_total['rawcensustractandblock'].describe()

In [ ]:
df_total['censustractandblock'].describe()

In [ ]:
float(df_total['rawcensustractandblock'].head(1))

In [ ]:
float(df_total['censustractandblock'].head(1))

In [ ]:
float(df_total['censustractandblock'].max())

The format of these values is FIPS code + census tract(six digits, in the format XXXX.XX for the raw version) + census block (four digits). The "raw" value appears to have three extra digits and censustractandblock rounds off those digits, but it also has incorrect values (eg the max has an incorrect FIPS code). 

For the US census, a county is divided into tracts, which is divided into blocks (which can be grouped together based on the first digit). A census block is the smallest geographic unit for which census data is collected, and their numbers are out of 9999 per census tract. If we wanted to examine how neighborhood demographics affect the logerror, we would be able to match census data to groups of properties based on these values. 

In [ ]:
df_total['rawcensustractandblock'].fillna(0, inplace = True)
def census(a): 
    return int(round(a * 1000000))

df_total['rawcensustractandblock'] = df_total['rawcensustractandblock'].map(lambda a: census(a))

# Data Distributions

In [ ]:
categorical=['airconditioningtypeid','architecturalstyletypeid','buildingclasstypeid',
             'decktypeid','heatingorsystemtypeid','storytypeid','typeconstructiontypeid',
             'pooltype','propertycountylandusecode','propertylandusetypeid','propertyzoningdesc']
numerical = ['basementsqft','bathroomcnt','bedroomcnt','buildingqualitytypeid','calculatedbathnbr',
             'finishedfloor1squarefeet','calculatedfinishedsquarefeet','finishedsquarefeet12',
             'finishedsquarefeet13','finishedsquarefeet15','finishedsquarefeet50','finishedsquarefeet6',
             'fireplacecnt','fullbathcnt','garagecarcnt','garagetotalsqft','latitude','longitude',
             'lotsizesquarefeet','poolcnt','poolsizesum','pooltypeid10','pooltypeid2','pooltypeid7','roomcnt',
             'threequarterbathnbr','unitcnt','yardbuildingsqft17','yardbuildingsqft26','yearbuilt','numberofstories',
             'structuretaxvaluedollarcnt','taxvaluedollarcnt','assessmentyear','landtaxvaluedollarcnt','taxamount',
             'taxdelinquencyyear','logerror']
flags = ['hashottuborspa','fireplaceflag','taxdelinquencyflag',]


df_num=df_total[numerical]
df_cat=df_total[categorical]
df_flag=df_total[flags]

## Frequencies

### Log Error Frequencies

In [ ]:
plt.close('all')
fig,ax=plt.subplots(figsize=(7, 7))
ax.set(yscale="symlog")
g=sns.distplot(df_num['logerror'].values, bins=50, kde=False)
plt.show()

### Numerical Data Frequencies

In [ ]:
plt.close('all')
df_num.hist(figsize=(20, 20), bins=50, xlabelsize=8, ylabelsize=8);
plt.show()

### Categorical Data Distribution

We can plot categorical data together but need to group them based on similar axis sizes

In [ ]:
plt.close('all')
for c in categorical:
    sns.countplot(x=df_cat[c], data=df_cat, palette="Greens_d")
    plt.figure(figsize=(20,20))
    plt.show()

In [ ]:
df_flag.describe()

## Log Error Over Time

Let's look at the distribution of log errors over time:

In [ ]:
means = df_total.groupby('transactiondate')['logerror'].mean()

plt.close('all')
plt.figure(figsize=(20,5))
plt.scatter(df_total['transactiondate'].tolist(), df_total['logerror'], s =10, c = 'blue')
plt.scatter(means.index, means, s =10, c = 'red')
plt.title('LogError Over Time')
plt.xlabel('Transaction Date')
plt.ylabel('Logerror')
plt.show()
plt.close()

In [ ]:
df_total.groupby('setyear')['logerror'].describe()

The log error distributions are roughly consistent over time, with the annual means within one standard deviation of each other and an expected decrease in quantity during the winter months since there are fewer properties sold at that time of year.

## Correlations

In [ ]:
plt.close('all')
plt.figure(figsize=(12,12))
sns.heatmap(df_total[numerical].corr(), vmax=.8, square=True, cmap=cm.coolwarm)
plt.show()

We can see a slight positive correlation between basementsqft and logerror, as well as a slight negative correlation between taxdelinquencyyear and logerror.

More importantly, certain features are strongly correlated with eact other, consistent with the fact that they have the same or similar definitions in the data dictionary. In particular:
* bathroomcnt, calcualtedbathnbr, fullbathcnt
* finishedfloor1squarefeet, finishedsquarefeet50
* calculatedfinishedsquarefeet, finishedsquarefeet12, finsihedsquarefeet13, finishedsquarefeet15, finishedsquarefeet6
* structuretaxvaluedollarcnt, taxvaluedollarcnt, landvaluedollarcnt, taxamount

Certain features are also correlated that logically would reflect the size of the house - including number of bathrooms, number of bedrooms, total square footage, and the structure tax value.

# Missing Data

## Percent Missing Data

In [ ]:
missing_percents16 = (len(df_merged16.index) - df_merged16.count())/len(df_merged16.index)
missing_percents17 = (len(df_merged17.index) - df_merged17.count())/len(df_merged17.index)

In [ ]:
missing_percents16.sort_values(inplace=True)
temp = pd.DataFrame(missing_percents17, columns=['2017'])
missing_combined = pd.DataFrame(missing_percents16, columns=['2016'])
missing_combined = missing_combined.join(temp)

In [ ]:
missing_combined.plot.barh(figsize=(20,40))
plt.yticks(size=20)
plt.show()

# What Features Both Over and Underestimate the Log Error?